# Omnipose Training and Retraining

This Notebook is going to be used to set up a training environment for omnipose on the Nikon computer with GPU activated. 

*1. Train Using only our images*

*2. Train Using our Images and Selected Images From Original Omnipose Dataset*

*3. Retrain Using our Images.*

I am going to try to clock the training time for each of these to udnerstand what is really required. 


#### Imports

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


In [2]:
from pathlib import Path
from cellpose_omni import plot, models, core, io, dynamics
import omnipose, ncolor
import skimage.io
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import time, os, sys
from tifffile import imread
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
from stardist import fill_label_holes
from cellpose_omni.models import CellposeModel


c:\Users\Nikon\anaconda3\envs\omnipose\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\Nikon\anaconda3\envs\omnipose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-10-31 14:24:23,190 [INFO] ** TORCH GPU version installed and working. **
>>> GPU activated? 1


## Data Loading


In [29]:
from pathlib import Path

# Define the base directory and file extensions
basedir = 'C:/Users/Nikon/Downloads/Omni/omni_retrain/'
path = 'C:/Users/Nikon/Downloads/Omni/omni_retrain/'
img_extension = "*.tif"
mask_extension = "*_cp_masks.png"
imgs_files = [str(p) for p in Path(path).glob("*.tif")]
lbls_files  = [str(p) for p in Path(path).glob("*.png")]
# Get image and mask file paths
img_names = [str(p) for p in Path(basedir).glob(f"**/{img_extension}")]
mask_names = [str(p) for p in Path(basedir).glob(f"**/{mask_extension}")]
imgs_files = glob(os.path.join(path, "Image*.tiff"))
lbls_files = [f.replace("Image", "segm").replace("tiff", "tif") for f in imgs_files]

NameError: name 'glob' is not defined

In [22]:
# Checking to make sure I have the files correctly

img_basenames = [os.path.basename(os.path.splitext(name)[0]) for name in img_names]
mask_basenames = [os.path.basename(os.path.splitext(name)[0]).replace('_cp_masks', '') for name in mask_names]

matching_pairs = set(img_basenames).intersection(set(mask_basenames))

is_img_sorted = img_names == sorted(img_names)
is_mask_sorted = mask_names == sorted(mask_names)

print(len(img_names))
print(len(mask_names))
print((img_names))
print((mask_names))
print(is_img_sorted)
print(is_mask_sorted)
print(matching_pairs)



12
12
['C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0001.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0002.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0004.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0006.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0007.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0008.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0009.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0010.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain\\20230812_LZ22225_10min_uninf.006\\phase-MAX-0011.tif', 'C:\\Users\\Nikon\\Downloads\\Omni\\omni_retrain

In [27]:
# Obviously sorting the masks by corresponding base name is critical. The different suffixes mess this up, but sorted()
# can take a function like the one below as a key. Modifying the default suffix requires the lambda syntax (e.g. cellpose). 
def getname(path,suffix='_masks'):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

mask_names = sorted(mask_names,key=getname)
img_names = sorted(img_names,key=getname)
imgs = [skimage.io.imread(f) for f in img_names]
lbls = [ncolor.format_labels(omnipose.utils.clean_boundary(skimage.io.imread(f))) for f in mask_names]

imgs = [skimage.io.imread(f)[np.newaxis, ...] for f in imgs_files]
lbls = [fill_label_holes(skimage.io.imread(f)) for f in lbls_files]
print("imgs: ", [i.shape for i in imgs])
print("lbls: ", [i.shape for i in lbls])

model = CellposeModel(
    gpu=True, omni=True, nclasses=4, nchan=1, diam_mean=0,
)

NameError: name 'imgs_files' is not defined

## Ok Good checks, now hweres how you run

To run the training command via CLI, you have several options, but the general idea is to open a terminal window where Python and the required libraries are accessible. Here's a step-by-step guide tailored to your specific needs:

### Using Anaconda Terminal (Recommended for Anaconda Users)

1. **Open Anaconda Navigator**: This is the GUI that Anaconda provides. From here, you can launch various applications, including terminals with specific environments.

2. **Launch Terminal**: Navigate to the "Environments" tab and locate the environment where you have installed the relevant packages (e.g., `omnipose`, `cellpose`, etc.). Click on the "play" button next to it and choose "Open Terminal."

3. **Navigate to Working Directory**: Use the `cd` command to navigate to the directory where your data is stored. For example:
    ```bash
    cd "C:/Users/Nikon/Downloads/Omni/omni_retrain/"
    ```

4. **Run Command**: Paste and run the training command you've constructed.

### Using Windows CMD

1. **Open Command Prompt**: You can do this by searching for "cmd" in the Windows search bar.

2. **Activate Anaconda Environment**: If you're using Anaconda, you'll need to activate the environment where you've installed the required packages. The command is usually:
    ```bash
    conda activate your_environment_name
    ```

3. **Navigate to Working Directory**: Similar to the Anaconda terminal, use the `cd` command.

4. **Run Command**: Paste and run the training command.

### Using a `.py` File (Optional)

If you find yourself running this command frequently, you might want to save it in a Python script (`*.py`) file. This way, you can simply run the script each time, rather than pasting the command.

1. **Create a `.py` File**: You can do this in any text editor. Save it in the directory where your data is stored.

2. **Paste Command**: Convert the CLI command into Python code using the `os.system()` function from the `os` library. For example:
    ```python
    import os

    os.system("your_entire_command_here")
    ```

3. **Run Script**: Navigate to the directory where the `.py` file is stored and run:
    ```bash
    python your_script_name.py
    ```

Choose the method that you find most convenient. The Anaconda terminal is often the easiest if you're using an Anaconda-managed Python environment, as it ensures that all the dependencies are correctly set up.

In [ ]:
omnipose --train --use_gpu --dir "C:/Users/Nikon/Downloads/Omni/omni_retrain/" \
         --img_filter ".tif" --mask_filter "_cp_masks.png" \
         --nchan 1 --all_channels --channel_axis 0 \
         --pretrained_model None --diameter 0 --nclasses 3 \
         --learning_rate 0.1 --RAdam --batch_size 16 --n_epochs 400 \
         --look_one_level_down


In [23]:
n = [0, 1]
model.train(
  [imgs[i] for i in n], [lbls[i] for i in n], 
  save_every=1, 
  n_epochs=400, 
  batch_size=16, 
  save_path=os.path.join(basedir, 'newtrain_fishonly'),
  channels=[0, 0], 
  min_train_masks=1, 
  rescale=0
)


NameError: name 'model' is not defined

In [1]:
!echo $KMP_DUPLICATE_LIB_OK

$KMP_DUPLICATE_LIB_OK
